Preprocess data for curriculum learnining approach

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import prepocess_functs_CL as pp
from os import listdir
from os.path import isfile, join

In [3]:
nticks = 200
ADC_MIN = 0
path = '/home/vlian/Workspace/ar39-samples-full/'
wireplane = 'U'

In [4]:
# data loader
def get_data(wireplane, path):
    noise_path = '../../../../train_dune_lartpc_v2/noise/'
    print('ar39: ', path)
    print('noise: ', noise_path)
    print('')
    print('----------loading----------')
    sig_name = wireplane+"-signal"
    cln_name = wireplane+"-clnsig"
    
    filenames1 = sorted([path+f for f in listdir(path) if (isfile(join(path, f)) and sig_name in f)])
    clean_filenames1 = sorted([path+f for f in listdir(path) if (isfile(join(path, f)) and cln_name in f)])
    #filenames =  filenames1[:int(len(filenames1)/10)]
    filenames =  filenames1
    #clean_filenames = clean_filenames1[:int(len(clean_filenames1)/10)]
    clean_filenames = clean_filenames1
    noise_filenames = sorted([f for f in listdir(noise_path) if (isfile(join(noise_path, f)) and wireplane in f)])

    combined_data = np.concatenate([np.load(fname, mmap_mode='r') for fname in filenames])
    combined_clean_data = np.concatenate([np.load(fname, mmap_mode='r') for fname in clean_filenames])
    combined_noise = np.concatenate([np.load(noise_path+fname, mmap_mode='r') for fname in noise_filenames])
    print('--------data loaded!-------')

    return combined_data, combined_clean_data, combined_noise
    #return combined_clean_data, combined_noise

Load data, filter of signal waveforms with ADC <= 3

# PLANE U

In [5]:
# load raw data
combined_data, combined_clean_data, combined_noise = get_data(wireplane, path)
#combined_clean_data, combined_noise = get_data(wireplane, path)
print('---------------------------------')
#print('     signal+noise: ', len(combined_data))
print('     clean signal: ', len(combined_clean_data))
print('     noise       : ', len(combined_noise))
print('---------------------------------')

# extract waveforms
signal_waveforms = pp.get_std_waveforms(combined_data, nticks)
clean_signal_waveforms = pp.get_std_waveforms(combined_clean_data, nticks)  # for autoencoder
print('')
print('filtering out small signals --> ADC >', ADC_MIN)
#print('     noise+signal : ', signal_waveforms.shape)
print('     clean signal : ', clean_signal_waveforms.shape)
#Filter out tiny signals < ADC_MIN, but leave big signals to test on (incl > ADC_MAX)
signal_waveforms, clean_signal_waveforms = pp.filter_signal_ADC_min(signal_waveforms,
                                            clean_signal_waveforms, ADC_MIN)

#_, clean_signal_waveforms = pp.filter_signal_ADC_min(clean_signal_waveforms,
#                                            clean_signal_waveforms, ADC_MIN)
print('------------after filtering------------')
print('     noise+signal : ', signal_waveforms.shape)
print('     clean signal : ', clean_signal_waveforms.shape)

grouped_waves= pp.adc_grouping(signal_waveforms, clean_signal_waveforms)

ar39:  /home/vlian/Workspace/ar39-samples-full/
noise:  ../../../../train_dune_lartpc_v2/noise/

----------loading----------


: 

: 

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.title('Number of signal waves grouped by ADC ranges', size=15)

x_labels = [pp.group_num_labels[i] for i in range(8)]
bar_width = 0.2

plt.bar([i for i in range(7)], grouped_waves[1][0:7], width=bar_width, color='blue', label='Induction Plane (U)')

for i in range(7):
    dist = 1000
    plt.text(i, grouped_waves[1][i] + dist, grouped_waves[1][i], color='blue')


plt.xticks([i for i in range(7)], x_labels, size=12)
plt.yticks(size=12)
plt.legend()
plt.xlabel('ADC Range', size=15)
plt.ylabel('Count', size=15)
plt.show()